#0.環境構築

##0-1.まずは以下のサイトよりDeepInfraのアカウントを作成してください。

https://deepinfra.com/

##0-2.以下よりAPI KEYを取得してセルにセットしてください。

https://deepinfra.com/dash/api_keys

In [ ]:
DEEPINFRA_API_KEY = "eeS07GRd50jqPKjdtINo6jBfcPXS2tig"

##0-3.Google Driveをマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


エクセルファイルの保存先パスを設定してください。

In [ ]:
G_FOLDER = "/content/drive/MyDrive/松尾研/geniac/データチーム/インストラクションデータ"

##0-4.以下のコードで必要なライブラリのインストールと、APIのテスト実行を行います。

In [ ]:
!pip install -qU openai datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.3/320.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00


In [ ]:
# Assume openai>=1.0.0
from openai import OpenAI

# Create an OpenAI client with your deepinfra token and endpoint
openai = OpenAI(
    api_key=DEEPINFRA_API_KEY,
    base_url="https://api.deepinfra.com/v1/openai",
)

In [ ]:
chat_completion = openai.chat.completions.create(
    model="mistralai/Mixtral-8x22B-Instruct-v0.1",
    messages=[{"role": "system", "content": "あなたは日本語で回答するAIボットです。"},
     {"role": "user", "content": "以下の文章が文法的に正しいかどうかを評価してください。文法的に正しい場合には1を、誤りを含んでいる場合には0を選択してください。  本件に関しましては、貴社のご要望を踏まえて最適なソリューションをご提案させていただきます次第でございます。"}],
)

print(chat_completion.choices[0].message.content)

 回答:\1

文法的には正しい文章です。


#1.データ生成処理の実行

##1-1.プロンプト準備

言語理解でのプロンプト例

In [ ]:
PROMPT_TEMPLATE = """
以下のタスク指示例を参考に、10種類の多様な質問文（input）の文のペアと回答文（output）のスコアを生成してください。

要件は以下の通りです：
1. 各文で動詞を繰り返さないようにしてください。
2. 質問文は、多様なビジネスシーンを想定してください。
3. ビジネスで使用する言葉遣いで回答してください。
4. です・ます調で回答してください。
5. 全ての文は100語以内にしてください。
6. タスク指示例のとおり、"id", "instruction", "input", "output"をJson形式で回答してください。
7. タスク指示例のとおり、"id"は1以上の連続した整数値にしてください。
8. タスク指示例のとおり、"instruction"は固定値にしてください。
9. タスク指示例のとおり、"input"は100語以内の文のペアにしてください。
10. タスク指示例のとおり、"output"のスコアは質問文（input）の文のペアの類似度をスコア基準に従って0.0~5.0の数値で評価してください。

スコア基準：
 4.5以上5.0以下: ほぼ同義
 3.5以上4.5未満: 関連が強いが、細かい点で異なる
 2.5以上3.5未満: 関連はあるが、重要な部分で異なる
 1.5以上2.5未満: 関連性は少しあるが、大部分が異なる
 0.5以上1.5未満: ほとんど関連性がない
 0.5未満0.0以上: 無関係

タスク指示例：
$FewShotExamples
"""

In [ ]:
FEWSHOT_EXAMPLES = [
  {
    "id": 1,
    "instruction": """以下の質問文（input）の文のペアの類似度を0.0から5.0までのスコアで評価して回答（output）してください。
スコアの基準は以下の通りです。
 4.5以上5.0以下: ほぼ同義
 3.5以上4.5未満: 関連が強いが、細かい点で異なる
 2.5以上3.5未満: 関連はあるが、重要な部分で異なる
 1.5以上2.5未満: 関連性は少しあるが、大部分が異なる
 0.5以上1.5未満: ほとんど関連性がない
 0.5未満0.0以上: 無関係""",
    "input": "彼は仕事で成功を収めるために懸命に働きました。彼は職場で良い結果を出すために一生懸命努力しました。",
    "output": 4.8
  },
  {
    "id": 2,
    "instruction": """以下の質問文（input）の文のペアの類似度を0.0から5.0までのスコアで評価して回答（output）してください。
スコアの基準は以下の通りです。
 4.5以上5.0以下: ほぼ同義
 3.5以上4.5未満: 関連が強いが、細かい点で異なる
 2.5以上3.5未満: 関連はあるが、重要な部分で異なる
 1.5以上2.5未満: 関連性は少しあるが、大部分が異なる
 0.5以上1.5未満: ほとんど関連性がない
 0.5未満0.0以上: 無関係""",
    "input": "彼女は食品会社で働いており、新規顧客の獲得に取り組んでいます。彼女は顧客拡大戦略の打ち立てに力を入れています。",
    "output": 3.6
  },
  {
    "id": 3,
    "instruction": """以下の質問文（input）の文のペアの類似度を0.0から5.0までのスコアで評価して回答（output）してください。
スコアの基準は以下の通りです。
 4.5以上5.0以下: ほぼ同義
 3.5以上4.5未満: 関連が強いが、細かい点で異なる
 2.5以上3.5未満: 関連はあるが、重要な部分で異なる
 1.5以上2.5未満: 関連性は少しあるが、大部分が異なる
 0.5以上1.5未満: ほとんど関連性がない
 0.5未満0.0以上: 無関係""",
    "input": "彼は2年前にベンチャー起業を立ち上げ、現在事業の多角化に取り組んでいます。彼は昨年度の実績が認められて、取締役専務から副社長に昇格しました。",
    "output": 0.2
  }
]

##1-2.データ生成処理の実行

パラメーターをセット

In [ ]:
TEMPERATURE=1 #temperature to use for sampling. 0 means the output is deterministic. Values greater than 1 encourage more diversity
MAX_TOKENS=10000
LOOP_NUM=1 #1回あたり10件データを生成

In [ ]:
def run_model(PROMPT_TEMPLATE, TEMPERATURE=1, MAX_TOKENS=10000):

    chat_completion = openai.chat.completions.create(
        model="mistralai/Mixtral-8x22B-Instruct-v0.1",
        messages=[{"role": "system", "content": "あなたは日本語の学習データセットを生成する専門家です。"},
         {"role": "user", "content": PROMPT_TEMPLATE}],
        temperature=TEMPERATURE,#temperature to use for sampling. 0 means the output is deterministic. Values greater than 1 encourage more diversity
        top_p=0.9,#0 < top_p ≤ 1 Sample from the set of tokens with highest probability such that sum of probabilies is higher than p. Lower values focus on the most probable tokens.Higher values sample more low-probability tokens
        max_tokens=MAX_TOKENS,
        frequency_penalty=1.3,#-2 ≤ presence_penalty ≤ 2 Positive values penalize new tokens based on how many times they appear in the text so far, increasing the model's likelihood to talk about new topics.
        presence_penalty=1.3 #-2 ≤ presence_penalty ≤ 2 Positive values penalize new tokens based on whether they appear in the text so far, increasing the model's likelihood to talk about new topics.
        )

    output = chat_completion.choices[0].message.content

    return output

In [ ]:
from string import Template
from datetime import datetime
import pandas as pd
import json

df_list = []

### few-shot template ###
formatted_prompt_template = Template(PROMPT_TEMPLATE).substitute(FewShotExamples=FEWSHOT_EXAMPLES)

for i in range(LOOP_NUM):

    ### JSON形式で出力が出てこないケースがあるため、3回までリトライ ###
    for retry_i in range(3):

        ### LLMによるデータ生成処理 ###
        output = run_model(formatted_prompt_template, TEMPERATURE, MAX_TOKENS)
        print(output)

        try:
            #output = json.loads(output)
            df_output = pd.read_json(output)
            df_list.append(df_output)
            print(f"Success parsing JSON")
            break
        except Exception as e:
            print(e)
            print(f"Error parsing JSON: {retry_i}")
            continue


df_final = pd.concat(df_list)

df_final.head()

 [{'id': 1, 'instruction': '以下の質問文（input）の文のペアの類似度を0.0から5.0までのスコアで評価して回答（output）してください。\nスコアの基準は以下の通りです。\n 4.5以上5.0以下: ほぼ同義\n 3.5以上4.5未満: 関連が強いが、細かい点で異なる\n 2.5以上3.5未満: 関連はあるが、重要な部分で異なる\n 1.5以上2.5未満: 関連性は少しあるが、大部分が異なる\n 0.5以上1.5未満: ほとんど関連性がない\n 0.5未満0.0以上: 無関係', 'input': '彼女はインフラ開発事業に就いており、国際市場への進出を計画しています。彼女は海外拠点の設立に向けた法的手続きを推進しています。', 'output': 4.3},
{'id': 2, 'instruction': '以下の質問文（input）の文のペアの類似度を0.0から5.0までのスコアで評価して回答（output）してください。\nスコアの基準は以下の通りです。\n 4.5以上5.0以下: ほぼ同義\n 3.5以上4.5未満: 関連が強いが、細かい点で異なる\n 2.5以上3.5未満: 関連はあるが、重要な部分で異なる\n 1.5以上2.5未満: 関連性は少しあるが、大部分が異なる\n 0.5以上1.5未満: ほとんど関連性がない\n 0.5未満0.0以上: 無関係', 'input': '彼はITサービス企業のマネージャーであり、 Organization's Goals の達成に向けた新規事業策定を行っています。彼は最近Organization's Strategies の決定に多大な影響力を与えました。', 'output': 4.0},
{'id': 3, 'instruction': '以下の質問文（input）の文のペアの類似度を0.0から5.0までのスコアで評価して回答（output）してください。\nスコアの基準は以下の通りです。\n 4.5以上5.0以下: ほぼ同義\n 3.5以上4.5未満: 関連が強いが、細かい点で異なる\n 2.5以上3.5未満: 関連はあるが、重要な部分で異なる\n 1.5以上2.5未満: 関連性は少しあるが、大部分が異なる\n 0.5以上1.5未満: ほとんど関連性がない\n 0.5未

ValueError: No objects to concatenate

##1-3.データの保存

In [ ]:
import datetime

# Get the current datetime
now = datetime.datetime.now()

# Format the datetime as a string
timestamp = now.strftime("%Y-%m-%d_%H-%M-%S")

# Create the filename with the timestamp
filename = f"/generated_data_{timestamp}.xlsx"

# Save the DataFrame to an Excel file with the timestamped filename
df_final.to_excel(G_FOLDER + filename, index=False)

print(f"File saved as: {filename}")